# Huấn luyện mô hình CNN cho phát hiện Steganography âm thanh

Notebook này thực hiện quá trình tải dữ liệu âm thanh, tiền xử lý (tạo Mel-spectrogram), áp dụng SpecAugment, xây dựng và huấn luyện mô hình CNN để phân loại âm thanh sạch và âm thanh giấu tin (stego). Cuối cùng, nó đánh giá mô hình và lưu lại các kết quả. Đây là phiên bản khởi tạo của mô hình.

## 1. Cài đặt và Imports thư viện

Trước khi chạy, đảm bảo bạn đã cài đặt tất cả các thư viện cần thiết. Bạn có thể chạy dòng sau nếu thiếu (bỏ dấu `#`):

In [ ]:
# !pip install numpy librosa matplotlib tensorflow scikit-learn


In [ ]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.metrics import Precision, Recall, AUC
from sklearn.metrics import precision_recall_curve, f1_score, confusion_matrix, classification_report
import random
import sys
import subprocess
print("Các thư viện đã được import thành công!")
try:
    import resampy  # type: ignore
except ImportError:
    print('Đang cài đặt resampy...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', 'resampy'])
    import resampy  # type: ignore
GLOBAL_SEED = 2025
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)
tf.random.set_seed(GLOBAL_SEED)
print(f'Đã thiết lập GLOBAL_SEED = {GLOBAL_SEED}')


## 2. Cấu hình GPU

Thiết lập cấu hình GPU để TensorFlow có thể sử dụng nếu có sẵn. Điều này giúp tăng tốc độ huấn luyện.

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print("Không tìm thấy GPU vật lý. TensorFlow sẽ chạy trên CPU.")


## 3. Cấu hình chung và Tham số

Thiết lập các hằng số và tham số quan trọng cho quá trình tiền xử lý dữ liệu và huấn luyện mô hình.

In [ ]:
# Đường dẫn đến dataset
DATA_PARENT_DIR = '/kaggle/input/data-lsb/data'

# Tên file processed data (lưu tại working dir)
PROCESSED_DATA_FILE = 'audio_data.npz'

SAMPLE_RATE = 22050
N_FFT = 2048
HOP_LENGTH = 512
N_MELS = 128
FIXED_DURATION_SEC = 4
EXPECTED_SPECTROGRAM_COLS = int(np.ceil(FIXED_DURATION_SEC * SAMPLE_RATE / HOP_LENGTH))

# Tham số cho mô hình
IMG_WIDTH, IMG_HEIGHT = EXPECTED_SPECTROGRAM_COLS, N_MELS
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 1)
BATCH_SIZE = 8
EPOCHS = 150

# --- ĐƯỜNG DẪN LƯU OUTPUT ---
WORKING_DIR = '/kaggle/working/'
MODEL_SAVE_DIR = os.path.join(WORKING_DIR, 'model_output')
RESULT_IMAGE_DIR = os.path.join(WORKING_DIR, 'result_images')

print('Các tham số cấu hình đã được đặt:')
print(f'  Kích thước spectrogram mong đợi: {IMG_HEIGHT}x{IMG_WIDTH}')
print(f'  Thư mục dataset gốc: {DATA_PARENT_DIR}')
print(f'  File dữ liệu xử lý: {PROCESSED_DATA_FILE}')
print(f'  Thư mục lưu model: {MODEL_SAVE_DIR}')
print(f'  Thư mục lưu hình ảnh: {RESULT_IMAGE_DIR}')



## 4. Hàm Augmentation (SpecAugment)

Định nghĩa các hàm `time_masking` và `frequency_masking` để thực hiện kỹ thuật SpecAugment trên Mel-spectrogram. Các hàm này sẽ tạo ra các mặt nạ thời gian và tần số ngẫu nhiên để tăng cường tính đa dạng của dữ liệu huấn luyện.

In [ ]:
def time_masking(spectrogram, T=20, num_masks=2, replace_with_zero=True, axis_to_mask=1):
    cloned = np.copy(spectrogram)
    len_frames = cloned.shape[axis_to_mask]

    for _ in range(num_masks):
        t = random.randint(0, T)
        if len_frames == 0: continue
        t0 = random.randint(0, len_frames - t) if len_frames > t else 0

        if replace_with_zero:
            if cloned.ndim == 3:
                cloned[:, t0:t0 + t, :] = 0
            else:
                cloned[:, t0:t0 + t] = 0
        else:
            mean_val = np.mean(cloned)
            if cloned.ndim == 3:
                cloned[:, t0:t0 + t, :] = mean_val
            else:
                cloned[:, t0:t0 + t] = mean_val
    return cloned

def frequency_masking(spectrogram, F=15, num_masks=2, replace_with_zero=True, axis_to_mask=0):
    cloned = np.copy(spectrogram)
    num_mels = cloned.shape[axis_to_mask]

    for _ in range(num_masks):
        f = random.randint(0, F)
        if num_mels == 0: continue
        f0 = random.randint(0, num_mels - f) if num_mels > f else 0

        if replace_with_zero:
            if cloned.ndim == 3:
                cloned[f0:f0 + f, :, :] = 0
            else:
                cloned[f0:f0 + f] = 0
        else:
            mean_val = np.mean(cloned)
            if cloned.ndim == 3:
                cloned[f0:f0 + f, :, :] = mean_val
            else:
                cloned[f0:f0 + f] = mean_val
    return cloned

print("Các hàm SpecAugment đã được định nghĩa.")


## 5. Keras Sequence cho Data Augmentation

Class `SpecAugmentSequence` kế thừa từ `tf.keras.utils.Sequence` để cung cấp dữ liệu theo từng batch cho Keras, đồng thời áp dụng SpecAugment (time masking và frequency masking) trực tiếp trong quá trình huấn luyện.

In [ ]:
class SpecAugmentSequence(Sequence):
    def __init__(self, x_set, y_set, batch_size,
                 apply_time_mask=True, time_mask_param_T=20, num_time_masks=2,
                 apply_freq_mask=True, freq_mask_param_F=15, num_freq_masks=2,
                 augment_prob=0.6):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.apply_time_mask = apply_time_mask
        self.time_mask_param_T = time_mask_param_T
        self.num_time_masks = num_time_masks
        self.apply_freq_mask = apply_freq_mask
        self.freq_mask_param_F = freq_mask_param_F
        self.num_freq_masks = num_freq_masks
        self.augment_prob = augment_prob
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = self.x[batch_x_indices]
        batch_y = self.y[batch_x_indices]

        augmented_batch_x = []
        for spec_idx in range(batch_x.shape[0]):
            spec = batch_x[spec_idx]
            
            augmented_spec = np.copy(spec)
            if random.random() < self.augment_prob:
                if self.apply_time_mask:
                    augmented_spec = time_masking(augmented_spec, T=self.time_mask_param_T, num_masks=self.num_time_masks, axis_to_mask=1)
                if self.apply_freq_mask:
                    augmented_spec = frequency_masking(augmented_spec, F=self.freq_mask_param_F, num_masks=self.num_freq_masks, axis_to_mask=0)
            augmented_batch_x.append(augmented_spec)

        return np.array(augmented_batch_x), batch_y

    def on_epoch_end(self):
        self.indices = np.arange(len(self.x))
        np.random.shuffle(self.indices)


## 6. Hàm tải và tiền xử lý dữ liệu

Các hàm này chịu trách nhiệm đọc file âm thanh, chuyển đổi chúng thành Mel-spectrogram, chuẩn hóa kích thước và gán nhãn. Sau đó, dữ liệu sẽ được lưu vào file `.npz` để sử dụng lại mà không cần xử lý lại từ đầu.

In [ ]:
def load_and_preprocess_audio(file_path, sr=SAMPLE_RATE, duration=FIXED_DURATION_SEC, n_mels=N_MELS, n_fft=N_FFT, hop_length=HOP_LENGTH):
    try:
        samples, sr_orig = librosa.load(file_path, sr=sr, duration=duration, res_type='kaiser_fast')
        if len(samples) < duration * sr:
            samples = librosa.util.fix_length(samples, size=duration * sr)

        mel_spectrogram = librosa.feature.melspectrogram(y=samples, sr=sr_orig, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

        if log_mel_spectrogram.shape[1] < EXPECTED_SPECTROGRAM_COLS:
            pad_width = EXPECTED_SPECTROGRAM_COLS - log_mel_spectrogram.shape[1]
            log_mel_spectrogram = np.pad(log_mel_spectrogram, ((0, 0), (0, pad_width)), mode='constant')
        elif log_mel_spectrogram.shape[1] > EXPECTED_SPECTROGRAM_COLS:
            log_mel_spectrogram = log_mel_spectrogram[:, :EXPECTED_SPECTROGRAM_COLS]
        return log_mel_spectrogram
    except Exception as e:
        print(f"Lỗi khi xử lý file {file_path}: {e}")
        return None

def create_dataset_from_paths(data_paths_labels):
    features = []
    labels = []
    total_files = len(data_paths_labels)
    print(f"Bắt đầu tạo dataset từ {total_files} file...")
    for i, (file_path, label) in enumerate(data_paths_labels):
        spectrogram = load_and_preprocess_audio(file_path)
        if spectrogram is not None:
            features.append(spectrogram)
            labels.append(label)
        if (i + 1) % 200 == 0 or (i + 1) == total_files:
            print(f"  Đã xử lý {i+1}/{total_files} file.")
    if not features:
        print("Không có features nào được tạo.")
        return np.array([]), np.array([])
    features = np.array(features)
    labels = np.array(labels)
    if features.ndim == 3 and features.size > 0:
        features = features.reshape(features.shape[0], IMG_HEIGHT, IMG_WIDTH, 1)
    print(f"Hoàn tất tạo dataset. Số lượng features: {features.shape}, Số lượng labels: {labels.shape}")
    return features, labels

def reconstruct_file_lists(base_dir):
    sets = ['train', 'val', 'test']
    all_data = {'train': [], 'val': [], 'test': []}
    for s in sets:
        clean_dir = os.path.join(base_dir, 'clean', s)
        stego_dir = os.path.join(base_dir, 'stego', s)

        if os.path.exists(clean_dir):
            for filename in os.listdir(clean_dir):
                if filename.lower().endswith('.wav'):
                    all_data[s].append((os.path.join(clean_dir, filename), 0))
        else:
            print(f"CẢNH BÁO: Thư mục file sạch không tồn tại: {clean_dir}")

        if os.path.exists(stego_dir):
            for filename in os.listdir(stego_dir):
                if filename.lower().endswith('.wav'):
                    all_data[s].append((os.path.join(stego_dir, filename), 1))
        else:
            print(f"CẢNH BÁO: Thư mục file stego không tồn tại: {stego_dir}")

        random.shuffle(all_data[s])

    print(f"Đã tái tạo danh sách file: Train ({len(all_data['train'])}), Val ({len(all_data['val'])}), Test ({len(all_data['test'])}).")
    return all_data['train'], all_data['val'], all_data['test']

def prepare_and_save_data(data_parent_dir, output_filepath):
    print("--- Bước 1 & 2: Tải, Tiền xử lý Dữ liệu và Lưu trữ ---")
    final_train_data, final_val_data, final_test_data = reconstruct_file_lists(data_parent_dir)

    if not final_train_data:
        print("LỖI: Không có dữ liệu huấn luyện nào được tìm thấy.")
        return None, None, None, None, None, None

    X_train, y_train = create_dataset_from_paths(final_train_data)

    X_val, y_val = np.array([]), np.array([])
    if final_val_data:
        X_val, y_val = create_dataset_from_paths(final_val_data)

    X_test, y_test = np.array([]), np.array([])
    if final_test_data:
        X_test, y_test = create_dataset_from_paths(final_test_data)

    if X_train.size == 0:
        print("LỖI: Không tạo được features cho tập huấn luyện.")
        return None, None, None, None, None, None

    print(f"--- Đang lưu dữ liệu đã xử lý vào '{output_filepath}' ---")
    data_to_save = {'X_train': X_train, 'y_train': y_train}
    if X_val.size > 0:
        data_to_save['X_val'] = X_val
        data_to_save['y_val'] = y_val
    if X_test.size > 0:
        data_to_save['X_test'] = X_test
        data_to_save['y_test'] = y_test

    np.savez_compressed(output_filepath, **data_to_save)
    print("--- Lưu dữ liệu thành công ---")
    return X_train, y_train, X_val, y_val, X_test, y_test

print("Các hàm tiền xử lý dữ liệu đã được định nghĩa.")



## 7. Hàm xây dựng mô hình CNN

Định nghĩa kiến trúc của mô hình CNN. Mô hình bao gồm các lớp tích chập (Conv2D), BatchNormalization, MaxPooling, Dropout và các lớp Dense.

In [ ]:
def build_cnn_model(input_shape):
    model = Sequential()
    l2_rate = 0.0015

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same',
                     kernel_regularizer=regularizers.l2(l2_rate)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(l2_rate)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.35))

    model.add(Conv2D(96, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(l2_rate)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())

    model.add(Dense(128, activation='relu',
                    kernel_regularizer=regularizers.l2(l2_rate)))
    model.add(BatchNormalization())
    model.add(Dropout(0.55))

    model.add(Dense(1, activation='sigmoid'))

    model.summary()
    return model

print("Hàm build_cnn_model đã được định nghĩa.")


## 8. Chuẩn bị thư mục và tải/xử lý dữ liệu

Ở bước này, chúng ta sẽ kiểm tra và tạo các thư mục cần thiết, sau đó cố gắng tải dữ liệu đã tiền xử lý từ file `.npz`. Nếu file không tồn tại hoặc bị lỗi, chương trình sẽ tự động chạy lại quá trình tiền xử lý từ các file `.wav` gốc và lưu lại.

In [ ]:
# Tạo thư mục nếu chưa tồn tại
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
os.makedirs(RESULT_IMAGE_DIR, exist_ok=True)

if not os.path.exists(DATA_PARENT_DIR):
    raise FileNotFoundError(
        f"Không tìm thấy DATA_PARENT_DIR: {DATA_PARENT_DIR}. Hãy kiểm tra lại đường dẫn dataset trên Kaggle."
    )

print(f"Notebook sẽ sử dụng file '{PROCESSED_DATA_FILE}' tại thư mục làm việc: {WORKING_DIR}\n")

processed_data_file_path = os.path.join(WORKING_DIR, PROCESSED_DATA_FILE)

X_train, y_train, X_val, y_val, X_test, y_test = None, None, None, None, None, None

if os.path.exists(processed_data_file_path):
    print(f"--- Đang tải dữ liệu đã xử lý từ '{processed_data_file_path}' ---")
    try:
        data = np.load(processed_data_file_path)
        X_train = data['X_train']
        y_train = data['y_train']
        X_val = data.get('X_val', np.array([]))
        y_val = data.get('y_val', np.array([]))
        X_test = data.get('X_test', np.array([]))
        y_test = data.get('y_test', np.array([]))
        print('--- Tải dữ liệu thành công ---')
        if X_train.size == 0:
            raise ValueError('Dữ liệu huấn luyện rỗng.')
    except Exception as exc:
        print(f"Lỗi khi tải dữ liệu đã xử lý: {exc}. Sẽ tiến hành tạo lại dữ liệu.\n")
        X_train, y_train, X_val, y_val, X_test, y_test = prepare_and_save_data(
            DATA_PARENT_DIR,
            processed_data_file_path
        )
else:
    print(f"--- Không tìm thấy dữ liệu đã xử lý. Đang tạo mới tại '{processed_data_file_path}' ---\n")
    X_train, y_train, X_val, y_val, X_test, y_test = prepare_and_save_data(
        DATA_PARENT_DIR,
        processed_data_file_path
    )

if X_train is None or X_train.size == 0:
    raise RuntimeError('Không thể chuẩn bị dữ liệu huấn luyện. Dừng notebook.')


def ensure_channel_dimension(arr, target_channels=None):
    if arr is None or arr.size == 0:
        return arr
    if arr.ndim == 3:
        arr = np.expand_dims(arr, axis=-1)
    elif arr.ndim != 4:
        raise ValueError(f'Dữ liệu đầu vào phải có 3 hoặc 4 chiều, nhận được shape {arr.shape}')
    if target_channels is not None and arr.shape[-1] != target_channels:
        if arr.shape[-1] == 1 and target_channels == 3:
            arr = np.repeat(arr, target_channels, axis=-1)
        elif arr.shape[-1] == 3 and target_channels == 1:
            arr = arr.mean(axis=-1, keepdims=True, dtype=arr.dtype)
        else:
            raise ValueError(
                f'Không thể chuyển số kênh từ {arr.shape[-1]} sang {target_channels} với dữ liệu hiện có.'
            )
    return arr


X_train = ensure_channel_dimension(X_train)
if X_val.size > 0:
    X_val = ensure_channel_dimension(X_val, target_channels=X_train.shape[-1])
if X_test.size > 0:
    X_test = ensure_channel_dimension(X_test, target_channels=X_train.shape[-1])

INPUT_SHAPE = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

print(f"Kích thước X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Số kênh đầu vào sau chuẩn hóa: {INPUT_SHAPE[-1]}")
if X_val.size > 0:
    print(f"Kích thước X_val: {X_val.shape}, y_val: {y_val.shape}")
else:
    print('Không có tập validation.')
if X_test.size > 0:
    print(f"Kích thước X_test: {X_test.shape}, y_test: {y_test.shape}")
else:
    print('Không có tập test.')



## 9. Tạo Data Generator và Compile Model

Khởi tạo `SpecAugmentSequence` để cung cấp dữ liệu huấn luyện với augmentation. Sau đó, xây dựng mô hình CNN và compile nó với optimizer Adam, hàm lỗi `binary_crossentropy` và các metrics cần thiết.

In [ ]:
# --- THAM SỐ CHO SPEC AUGMENT ---
TIME_MASK_PARAM_T = 20
NUM_TIME_MASKS = 2
FREQ_MASK_PARAM_F = 15
NUM_FREQ_MASKS = 2
AUGMENT_PROB = 0.6

train_data_generator = SpecAugmentSequence(
    X_train, y_train, BATCH_SIZE,
    apply_time_mask=True, time_mask_param_T=TIME_MASK_PARAM_T, num_time_masks=NUM_TIME_MASKS,
    apply_freq_mask=True, freq_mask_param_F=FREQ_MASK_PARAM_F, num_freq_masks=NUM_FREQ_MASKS,
    augment_prob=AUGMENT_PROB
)
print("Đã tạo Data Generator với SpecAugment cho tập huấn luyện.")
print(f"  Time Masking: T={TIME_MASK_PARAM_T}, num_masks={NUM_TIME_MASKS}")
print(f"  Frequency Masking: F={FREQ_MASK_PARAM_F}, num_masks={NUM_FREQ_MASKS}")
print(f"  Augmentation Probability: {AUGMENT_PROB*100:.1f}%")

print("--- Xây dựng Mô hình CNN ---")

model = build_cnn_model(INPUT_SHAPE)

# Compile model với Learning Rate Schedule
if X_train.size > 0:
    steps_per_epoch = max(len(X_train) // BATCH_SIZE, 1)
    decay_steps_actual = steps_per_epoch * 10
    decay_rate_actual = 0.90

    initial_learning_rate = 1e-4
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=decay_steps_actual,
        decay_rate=decay_rate_actual,
        staircase=True
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    print(f"Sử dụng ExponentialDecay LR Schedule với initial_lr={initial_learning_rate}, decay_steps={decay_steps_actual}, decay_rate={decay_rate_actual}")
else:
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    print("CẢNH BÁO: X_train rỗng, sử dụng Adam LR mặc định (1e-4).")

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), AUC(name='auc')]
)
print("Đã compile model với LR schedule và các metrics.")



## 10. Huấn luyện Mô hình

Tiến hành huấn luyện mô hình sử dụng Data Generator. Các callbacks `EarlyStopping` và `ModelCheckpoint` được sử dụng để dừng huấn luyện sớm khi validation loss không cải thiện và lưu lại trọng số của mô hình tốt nhất.

In [ ]:
print("\n--- Bắt đầu Huấn luyện Mô hình ---")

model_id = 'audio_cnn'
model_checkpoint_filename = f'best_model_{model_id}.keras'
training_history_plot_filename = f'training_history_{model_id}.png'

model_checkpoint_path = os.path.join(MODEL_SAVE_DIR, model_checkpoint_filename)
training_history_plot_path = os.path.join(RESULT_IMAGE_DIR, training_history_plot_filename)

callbacks_list = [
    ModelCheckpoint(
        model_checkpoint_path,
        monitor='val_loss' if X_val.size > 0 else 'loss',
        save_best_only=True,
        verbose=1
    )
]
if X_val.size > 0:
    callbacks_list.insert(0, EarlyStopping(
        monitor='val_loss',
        patience=25,
        restore_best_weights=True,
        verbose=1
    ))

history = model.fit(
    train_data_generator,
    epochs=EPOCHS,
    validation_data=(X_val, y_val) if X_val.size > 0 else None,
    callbacks=callbacks_list
)

print('Huấn luyện mô hình hoàn tất.')


## 11. Đánh giá và Tìm ngưỡng tối ưu trên tập Validation

Sau khi huấn luyện, chúng ta sẽ tải lại mô hình tốt nhất (từ checkpoint) và sử dụng tập validation để tìm ra ngưỡng phân loại tối ưu (dựa trên F1-score). Ngưỡng này sau đó sẽ được áp dụng cho tập kiểm thử.

In [ ]:
\
best_model_to_evaluate = None
if os.path.exists(model_checkpoint_path):
    print(f"\n--- Tải lại model tốt nhất từ '{model_checkpoint_path}' để đánh giá ---")
    try:
        best_model_to_evaluate = tf.keras.models.load_model(model_checkpoint_path)
        print("Đã load lại model tốt nhất từ checkpoint.")
    except Exception as e:
        print(f"Không thể tải lại model tốt nhất từ checkpoint: {e}.")
        if 'model' in locals() and model is not None:
            print("Sử dụng model hiện tại sau khi fit (nếu có) để đánh giá.")
            best_model_to_evaluate = model
else:
    print("CẢNH BÁO: Không tìm thấy file checkpoint. Sử dụng model hiện tại sau khi fit để đánh giá.")
    best_model_to_evaluate = model

best_threshold_f1 = 0.5
if best_model_to_evaluate is not None and X_val.size > 0 and y_val.size > 0:
    print("\n--- Tìm Ngưỡng Quyết Định Tối Ưu trên Tập Validation ---")
    y_pred_proba_val = best_model_to_evaluate.predict(X_val)

    precisions_pr, recalls_pr, thresholds_pr = precision_recall_curve(y_val, y_pred_proba_val)

    f1_scores_val_pr = np.array([])
    if len(precisions_pr) > 1 and len(recalls_pr) > 1:
        f1_scores_val_pr = (2 * precisions_pr[:-1] * recalls_pr[:-1]) / (precisions_pr[:-1] + recalls_pr[:-1] + 1e-9)

    if f1_scores_val_pr.size > 0:
        best_f1_idx_val = np.argmax(f1_scores_val_pr)
        if best_f1_idx_val < len(thresholds_pr):
            best_threshold_f1 = thresholds_pr[best_f1_idx_val]
            print(f"  Ngưỡng tối ưu cho F1-score trên tập Validation: {best_threshold_f1:.4f}")
            print(f"  F1-score cao nhất tương ứng: {f1_scores_val_pr[best_f1_idx_val]:.4f}")
            print(f"  Precision tại ngưỡng này: {precisions_pr[best_f1_idx_val]:.4f}")
            print(f"  Recall tại ngưỡng này: {recalls_pr[best_f1_idx_val]:.4f}")
        else:
            print("  Lỗi khi xác định ngưỡng F1 tốt nhất (index out of bounds for thresholds_pr). Sử dụng ngưỡng mặc định 0.5.")
            best_threshold_f1 = 0.5
    else:
        print("  Không thể tính F1-scores trên tập Validation (mảng F1 rỗng hoặc PR curve không đủ điểm). Sử dụng ngưỡng mặc định 0.5.")

    plt.figure(figsize=(8, 6))
    plt.plot(recalls_pr, precisions_pr, marker='.', label='Precision-Recall Curve')
    if 'best_f1_idx_val' in locals() and best_f1_idx_val < min(len(recalls_pr), len(precisions_pr)) and f1_scores_val_pr.size > 0:
        label_text = f"Best F1 Val (Thresh={best_threshold_f1:.2f}, F1={f1_scores_val_pr[best_f1_idx_val]:.2f})"
        plt.scatter(recalls_pr[best_f1_idx_val], precisions_pr[best_f1_idx_val], marker='o', s=100, color='red', label=label_text)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve on Validation Set')
    plt.legend()
    plt.grid(True)
    pr_curve_path = os.path.join(RESULT_IMAGE_DIR, f"precision_recall_curve_val_{os.path.splitext(model_checkpoint_filename)[0]}.png")
    try:
        plt.savefig(pr_curve_path)
        print(f"  Đã lưu Precision-Recall curve vào: {pr_curve_path}")
        plt.close()
    except Exception as e_plot:
        print(f"  Lỗi khi lưu Precision-Recall curve: {e_plot}")
        plt.close()
else:
    print("Không có dữ liệu validation để tìm ngưỡng tối ưu, hoặc không load được model. Sử dụng ngưỡng mặc định 0.5.")



## 12. Đánh giá trên tập Test và hiển thị kết quả

Cuối cùng, mô hình sẽ được đánh giá trên tập kiểm thử (test set) bằng cách sử dụng ngưỡng tối ưu đã tìm được. Các báo cáo đánh giá như `classification_report`, `confusion_matrix` và đồ thị lịch sử huấn luyện sẽ được hiển thị và lưu lại.

In [ ]:
\
if best_model_to_evaluate is not None and X_test.size > 0 and y_test.size > 0:
    print("\n--- Đánh giá Mô hình trên Tập Kiểm thử (với ngưỡng đã chọn) ---")
    y_pred_proba_test = best_model_to_evaluate.predict(X_test)
    y_pred_test_custom_threshold = (y_pred_proba_test >= best_threshold_f1).astype(int)

    print(f"\nKết quả đánh giá trên tập kiểm thử với ngưỡng: {best_threshold_f1:.4f}")
    target_names = ['Lớp Sạch (0)', 'Lớp Stego (1)']
    try:
        print(classification_report(y_test, y_pred_test_custom_threshold, target_names=target_names, digits=4, zero_division=0))
    except ValueError as ve:
        print(f"Lỗi khi tạo classification report: {ve}")

    cm = confusion_matrix(y_test, y_pred_test_custom_threshold)
    print("Confusion Matrix trên tập Test:")
    print(cm)

    print("\nKết quả đánh giá từ model.evaluate (ngưỡng mặc định 0.5 của Keras):")
    eval_results = best_model_to_evaluate.evaluate(X_test, y_test, verbose=0)
    metric_names = best_model_to_evaluate.metrics_names
    for name, value in zip(metric_names, eval_results):
        print(f"  {name.capitalize()}: {value:.4f}")
else:
    print("\nKhông có dữ liệu kiểm thử để đánh giá hoặc không có model.")

try:
    if hasattr(history, 'history') and history.history:
        metrics_to_plot = ['accuracy', 'loss', 'precision', 'recall', 'auc']
        available_metrics = [m for m in metrics_to_plot if m in history.history or f'val_{m}' in history.history]

        if available_metrics:
            num_rows_plot = (len(available_metrics) + 1) // 2
            plt.figure(figsize=(15, 5 * num_rows_plot))
            plot_idx = 1
            for metric in available_metrics:
                has_train_metric = metric in history.history and history.history[metric]
                has_val_metric = f'val_{metric}' in history.history and history.history[f'val_{metric}']

                if has_train_metric or has_val_metric:
                    plt.subplot(num_rows_plot, 2, plot_idx)
                    if has_train_metric:
                        plt.plot(history.history[metric], label=f'Training {metric.capitalize()}')
                    if has_val_metric:
                        plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric.capitalize()}')

                    plt.title(metric.capitalize())
                    plt.xlabel('Epoch')
                    plt.ylabel(metric.capitalize())
                    plt.legend()
                    plt.grid(True)
                    plot_idx += 1

            plt.tight_layout()
            try:
                plt.savefig(training_history_plot_path)
                print(f"Đã lưu đồ thị lịch sử huấn luyện vào '{training_history_plot_path}'")
                plt.close()
            except Exception as e_plot_hist:
                print(f"Lỗi khi lưu đồ thị lịch sử huấn luyện: {e_plot_hist}")
                plt.close()
        else:
            print("Không có metric nào trong history để vẽ đồ thị.")
    else:
        print("History object không có dữ liệu để vẽ đồ thị.")
except Exception as e:
    print(f"Lỗi khi vẽ đồ thị lịch sử huấn luyện: {e}")

print("Hoàn tất quá trình huấn luyện và đánh giá.")

